In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import backend as keras
from sklearn.metrics import jaccard_score

In [ ]:
x = np.load('x_train_test.npy')
y = np.load('y_train_test.npy')
x_val=np.load('x_validation.npy')
y_val=np.load('y_validation.npy')

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 103)
y_train = np.reshape(y_train,(1424,256,256,1))
y_test = np.reshape(y_test,(356,256,256,1))

In [ ]:
print('Training Images(x_train) Shape: ',x_train.shape)
print('Training Labels(y_train) Shape: ',y_train.shape)
print('Testing Images(x_test) Shape: ',x_test.shape)
print('Testing Labels(y_test) Shape: ',y_test.shape)
print('Validation Images(x_test) Shape: ',x_val.shape)
print('Validation Labels(y_test) Shape: ',y_val.shape)

In [ ]:
def satellite(input_img):
  
  conv_1 = Conv2D(filters = 32,kernel_size=(3,3),strides = (1,1),padding = 'same',kernel_initializer='he_normal')(input_img)
  batch_conv1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv_1)
  activate_batch_conv1 = Activation('relu')(batch_conv1)

  conv_2 = Conv2D(filters = 32,kernel_size = (3,3),strides = (1,1),padding = 'same',kernel_initializer = 'he_normal')(activate_batch_conv1)
  batch_conv2 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv_2)
  activate_batch_conv2 = Activation('relu')(batch_conv2)
  
  
  ######### POOL_Start
  pool_1 = MaxPooling2D(pool_size = (2,2),strides= (2,2))(activate_batch_conv2)
  pool_1 = Dropout(0.025)(pool_1)  
  ######### Pool_End
  
  
  conv_3 = Conv2D(filters = 64,kernel_size = (3,3),strides = (1,1),padding = 'same',kernel_initializer='he_normal')(pool_1)
  batch_conv3 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv_3)
  activate_batch_conv3 = Activation('relu')(batch_conv3)

  conv_4 = Conv2D(filters = 64,kernel_size = (3,3),strides = (1,1),padding = 'same',kernel_initializer='he_normal')(activate_batch_conv3)
  batch_conv4 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv_4)
  activate_batch_conv4 = Activation('relu')(batch_conv4)
  
  
  ######### POOL_Start
  pool_2 = MaxPooling2D(pool_size = (2,2),strides= (2,2))(activate_batch_conv4)
  pool_2 = Dropout(0.025)(pool_2)  
  ######### POOL_End
  
  
  conv_5 = Conv2D(filters = 128,kernel_size = (3,3),strides = (1,1),padding = 'same',kernel_initializer='he_normal')(pool_2)
  batch_conv5 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv_5)
  activate_batch_conv5 = Activation('relu')(batch_conv5)

  conv_6 = Conv2D(filters = 128,kernel_size = (3,3),strides = (1,1),padding = 'same',kernel_initializer='he_normal')(activate_batch_conv5)
  batch_conv6 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv_6)
  activate_batch_conv6 = Activation('relu')(batch_conv6)
  
  
  ######### POOL_Start
  pool_3 = MaxPooling2D(pool_size = (2,2),strides= (2,2))(activate_batch_conv6)
  pool_3 = Dropout(0.025)(pool_3)
  ######### POOL_End
  
  
  conv_7 = Conv2D(filters = 256,kernel_size = (3,3),strides = (1,1),padding = 'same',kernel_initializer='he_normal')(pool_3)
  batch_conv7 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv_7)
  activate_batch_conv7 = Activation('relu')(batch_conv7)

  conv_8 = Conv2D(filters = 256,kernel_size = (3,3),strides = (1,1),padding = 'same',kernel_initializer='he_normal')(activate_batch_conv7)
  batch_conv8 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv_8)
  activate_batch_conv8 = Activation('relu')(batch_conv8)
  
  
  #########  POOL
  pool_4 = MaxPooling2D(pool_size = (2,2),strides= (2,2))(activate_batch_conv8)
  pool_4 = Dropout(0.025)(pool_4)
  #########  POOL
  
  
  conv_9 = Conv2D(filters = 512,kernel_size = (3,3),strides = (1,1),padding = 'same',kernel_initializer='he_normal')(pool_4)
  batch_conv9 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv_9)
  activate_batch_conv9 = Activation('relu')(batch_conv9)

  conv_10 = Conv2D(filters = 512,kernel_size = (3,3),strides = (1,1),padding = 'same',kernel_initializer='he_normal')(activate_batch_conv9)
  batch_conv10 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv_10)
  activate_batch_conv10 = Activation('relu')(batch_conv10)
  
  
  ################Upsample-1 Start
  
  up1 = UpSampling2D(size = (2,2),interpolation = 'bilinear')(activate_batch_conv10)
  up1_conv = Conv2D(filters = 256,kernel_size = (3,3),strides = (1,1),padding = 'same',kernel_initializer='he_normal')(up1)
  merge_up1 = concatenate([activate_batch_conv8,up1_conv])
  drop_up1 = Dropout(0.025)(merge_up1)
  
  conv_11 = Conv2D(filters = 256,kernel_size = (3,3),strides = (1,1),padding = 'same',kernel_initializer='he_normal')(drop_up1)
  batch_conv11 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv_11)
  activate_batch_conv11 = Activation('relu')(batch_conv11)
  
  conv_12 = Conv2D(filters = 256,kernel_size = (3,3),strides = (1,1),padding = 'same',kernel_initializer='he_normal')(activate_batch_conv11)
  batch_conv12 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv_12)
  activate_batch_conv12 = Activation('relu')(batch_conv12)
  
  ################Upsample-1 End
  
  
  ################Upsample-2 Start
  
  up2 = UpSampling2D(size = (2,2),interpolation = 'bilinear')(activate_batch_conv12)
  up2_conv = Conv2D(filters = 128,kernel_size = (3,3),strides = (1,1),padding = 'same',kernel_initializer='he_normal')(up2)
  merge_up2 = concatenate([activate_batch_conv6,up2_conv])
  drop_up2 = Dropout(0.025)(merge_up2)
  
  conv_13 = Conv2D(filters = 128,kernel_size = (3,3),strides = (1,1),padding = 'same',kernel_initializer='he_normal')(drop_up2)
  batch_conv13 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv_13)
  activate_batch_conv13 = Activation('relu')(batch_conv13)
  
  conv_14 = Conv2D(filters = 128,kernel_size = (3,3),strides = (1,1),padding = 'same',kernel_initializer='he_normal')(activate_batch_conv13)
  batch_conv14 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv_14)
  activate_batch_conv14 = Activation('relu')(batch_conv14)
  
  ################Upsample-2 End
  
  
  ################Upsample-3 Start
  
  up3 = UpSampling2D(size = (2,2),interpolation = 'bilinear')(activate_batch_conv14)
  up3_conv = Conv2D(filters = 64,kernel_size = (3,3),strides = (1,1),padding = 'same',kernel_initializer='he_normal')(up3)
  merge_up3 = concatenate([activate_batch_conv4,up3_conv])
  drop_up3 = Dropout(0.025)(merge_up3)
  
  conv_15 = Conv2D(filters = 64,kernel_size = (3,3),strides = (1,1),padding = 'same',kernel_initializer='he_normal')(drop_up3)
  batch_conv15 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv_15)
  activate_batch_conv15 = Activation('relu')(batch_conv15)
  
  conv_16 = Conv2D(filters = 64,kernel_size = (3,3),strides = (1,1),padding = 'same',kernel_initializer='he_normal')(activate_batch_conv15)
  batch_conv16 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv_16)
  activate_batch_conv16 = Activation('relu')(batch_conv16)

  ################Upsample-3 End
  
  
  ################Upsample-4 Start
  
  up4 = UpSampling2D(size = (2,2),interpolation = 'bilinear')(activate_batch_conv16)
  up4_conv = Conv2D(filters = 32,kernel_size = (3,3),strides = (1,1),padding = 'same',kernel_initializer='he_normal')(up4)
  merge_up4 = concatenate([activate_batch_conv2,up4_conv])
  drop_up4 = Dropout(0.025)(merge_up4)
  
  conv_17 = Conv2D(filters = 32,kernel_size = (3,3),strides = (1,1),padding = 'same',kernel_initializer='he_normal')(drop_up4)
  batch_conv17 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv_17)
  activate_batch_conv17 = Activation('relu')(batch_conv17)
  
  conv_18 = Conv2D(filters = 32,kernel_size = (3,3),strides = (1,1),padding = 'same',kernel_initializer='he_normal')(activate_batch_conv17)
  batch_conv18 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv_18)
  activate_batch_conv18 = Activation('relu')(batch_conv18)
  
  conv_19 = Conv2D(filters = 2,kernel_size = (3,3),strides = (1,1),activation = 'relu',padding = 'same',kernel_initializer='he_normal')(activate_batch_conv18)
  conv_20 = Conv2D(filters = 1,kernel_size = (1,1),strides = (1,1),activation = 'sigmoid',padding = 'same',kernel_initializer='he_normal')(conv_19)

  ################Upsample-4 End
  model = Model(input = [input_img], output =[conv_20])
  return model
  



In [ ]:
def jaccard_distance_loss(y_true, y_pred, smooth=100):
    """
    Jaccard = (|X & Y|)/ (|X|+ |Y| - |X & Y|)
            = sum(|A*B|)/(sum(|A|)+sum(|B|)-sum(|A*B|))
    
    The jaccard distance loss is usefull for unbalanced datasets. This has been
    shifted so it converges on 0 and is smoothed to avoid exploding or disapearing
    gradient.
    
    Ref: https://en.wikipedia.org/wiki/Jaccard_index
    
    @url: https://gist.github.com/wassname/f1452b748efcbeb4cb9b1d059dce6f96
    @author: wassname
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth

In [ ]:
input_img = Input((256,256,3))
model = satellite(input_img)
#model.summary()
model.compile(optimizer=Adam(), loss=jaccard_distance_loss, metrics=["accuracy"])

In [ ]:
results = model.fit(x_train, y_train, batch_size=32, epochs=50,
                    validation_data=(x_test, y_test))

In [ ]:
res=model.predict(new_x_val)
print('IoU(Jaccard Score): ',round(jaccard_score(new_y_val.flatten(),res.flatten().round()),4))

In [ ]:
#model.save('/content/drive/My Drive/unet_v2.h5')